Import stdPackages manually:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, pandas as pd, numpy as np, pickle, math, pyDbs, scipy
from scipy import optimize, sparse, stats
from importlib.metadata import version
from IPython.display import IFrame
_repo = 'EnergyEconGroupWork'
d = {'main': os.path.join(os.getcwd().split(_repo,1)[0],_repo),
     'curr': os.getcwd()}
d.update({'py': os.path.join(d['main'], 'py'),
          'data': os.path.join(d['main'], 'Data')})
os.chdir(d['py'])
import base, lpCompiler, lpModels, mBasicPH_storage
os.chdir(d['curr'])

For working with the model, we first import some packages and the modelclass:

In [1]:
%run ../stdPackages.ipynb
from pyDbs import *
os.chdir(os.path.join(d['curr'],'EnergyEconGroupWork', 'py'))
from mBasicPH_storage import *
os.chdir(d['curr'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Next we load the database:

In [2]:
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'GeneratorsVariables','StorageVariables','TransmissionLines'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps', 'MarketMaps'],
        'variable2D': ['HourlyVariation']}
path_to_data = os.path.join(d['curr'],'EnergyEconGroupWork', 'Data', 'mBasicPH_storage_Data.xlsx')
db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

Convert variables to floats:

In [3]:
[db.__setitem__(var,db[var].astype(float).sort_index()) for var in db.getTypes('variable').keys()];

We need to read in heat areas manually:

In [4]:
db['g_H'] = db['g_E2g_H'].get_level_values('g_H')

Collect all areas in one:

In [5]:
db['g'] = db['g_E'].copy()

Finally, we will assume that linelosses and trade are zero such that the marginal willingness to pay and marginal generation costs of "transmission lines" does not have to be corrected. 

We then initialize and solve the model:

In [6]:
m = mSimple(db)
m()

c:\Users\mpher\anaconda3\envs\EE2023\Lib\site-packages\pyDbs\base.py:289: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  return x.sort_index().add(pd.Series(0, index = y).sort_index(),fill_value=fill_value) if (set(x_dom)-set(y_dom)) else pd.Series(0, index = y).sort_index().add(x.sort_index(),fill_value=fill_value)
c:\Users\mpher\anaconda3\envs\EE2023\Lib\site-packages\pyDbs\base.py:289: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  return x.sort_index().add(pd.Series(0, index = y).sort_index(),fill_value=fill_value) if (set(x_dom)-set(y_dom)) else pd.Series(0, index = y).sort_index().add(x.sort_index(),fill_value=fill_value)
c:\Users\mpher\anaconda3\envs\EE2023\Lib\site-packages\pyDbs\base.py:289: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  return x.sort_index().add(pd.Series(0, index = y).sort_index(),fill_value=f

ValueError: negative row index found

In [ ]:
db['g_H']